In [1]:
from tensorflow.keras.layers import Dense, Flatten, Input

from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

from glob import glob

import numpy as np

import matplotlib.pyplot as plt

In [2]:
# Load the pre-trained VGG16 model without the top (fully connected) layers
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))




58889256/58889256 [==============================] - 52s 1us/step


In [3]:
for layer in vgg.layers:
    layer.trainable=False
    
    

In [4]:
x=Flatten()(vgg.output)

In [5]:
#adding output layer
prediction=Dense(8,activation='softmax')(x)

In [6]:
#creating model
model = Model(inputs=vgg.input, outputs=prediction)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
#tell the model cost and optimization method to use

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'], 
    run_eagerly = True
    )

In [9]:
#import image data generator library
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

In [13]:
training_set = train_datagen.flow_from_directory('E:\\codings\\college-projects\\ResnetSkin\\skin-disease-datasaet\\train_set',
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size=20)

Found 920 images belonging to 8 classes.


In [15]:
test_set = train_datagen.flow_from_directory('E:\\codings\\college-projects\\ResnetSkin\\skin-disease-datasaet\\test_set',
                                           target_size=(224,224),
                                           class_mode='categorical',
                                           batch_size=20)

Found 233 images belonging to 8 classes.


In [16]:
training_set.class_indices

{'BA- cellulitis': 0,
 'BA-impetigo': 1,
 'FU-athlete-foot': 2,
 'FU-nail-fungus': 3,
 'FU-ringworm': 4,
 'PA-cutaneous-larva-migrans': 5,
 'VI-chickenpox': 6,
 'VI-shingles': 7}

In [17]:
#fit the model
r = model.fit_generator(training_set,
    validation_data=test_set,
    epochs=30,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
    )

C:\Users\Philemon Boaz\AppData\Local\Temp\ipykernel_16692\1968481864.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(training_set,


Epoch 1/30

46/46 [==============================] - 436s 10s/step - loss: 1.4952 - accuracy: 0.5293 - val_loss: 0.7663 - val_accuracy: 0.7425
Epoch 2/30
46/46 [==============================] - 254s 6s/step - loss: 0.5288 - accuracy: 0.8272 - val_loss: 0.6340 - val_accuracy: 0.7940
Epoch 3/30
46/46 [==============================] - 250s 5s/step - loss: 0.3625 - accuracy: 0.8793 - val_loss: 0.5601 - val_accuracy: 0.8240
Epoch 4/30
46/46 [==============================] - 250s 5s/step - loss: 0.2591 - accuracy: 0.9326 - val_loss: 0.4713 - val_accuracy: 0.8541
Epoch 5/30
46/46 [==============================] - 249s 5s/step - loss: 0.2241 - accuracy: 0.9435 - val_loss: 0.4738 - val_accuracy: 0.8670
Epoch 6/30
46/46 [==============================] - 477s 10s/step - loss: 0.1375 - accuracy: 0.9630 - val_loss: 0.4582 - val_accuracy: 0.9056
Epoch 7/30
46/46 [==============================] - 412s 9s/step - loss: 0.1406 - accuracy: 0.9696 - val_loss: 0.5233 - val_accuracy: 0.8841
Epoch 8/30

In [18]:
model.save('vgg16final.h5')

c:\Users\Philemon Boaz\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
